# Deltares Software Days 2022: Hydrolib-core demo

Hydrolib-core is a pip-installable python package. It consists of wrappers around the D-HYDRO model files.
Classes in Hydrolib-core are hierarchically organized by file type, and can be accessed via `hydrolib.core.io.<modeltype>.<filetype>.models`.
File objects in Hydrolib-core reflect the same structure as the file contents. 

In this demo, we will load a Flow FM model, make some changes to the model and save it again. 

## Step 0) Importing the modules 

In [ ]:
from hydrolib.core.io.mdu.models import FMModel
from hydrolib.core.io.structure.models import Weir, FlowDirection
from pathlib import Path

## Step 1) Loading a Flow FM model

In [ ]:
demo_data_folder = Path.cwd() / "data"
mdu_file_path = demo_data_folder /"FlowFM.mdu"

model = FMModel(filepath=mdu_file_path)
print(f"Loaded the model from {model.filepath}")

## Step 2) Inspecting the model

In [ ]:
# Show the hierachy tree of the model
model.show_tree()

# Get the structure file of the model. 
# Note that you can have multiple structure files, but this model has only one
assert model.geometry.structurefile is not None
structure_file = model.geometry.structurefile[0]

# Show the list of all the structures
print(f"Number of structures: {len(structure_file.structure)}")
print(structure_file.structure)

## Step 3) Adjusting the model

### Adding a new structure

In [ ]:
# Create a new weir.
weir = Weir(
    id="DV_1471", 
    branchId="458", 
    chainage=106.277467, 
    allowedFlowDir=FlowDirection.both, 
    crestLevel=0.400, 
    crestWidth=40.000, 
    corrCoeff=1.000, 
    useVelocityHeight=True
)

# It is also possible to create objects, by passing a dictionary.
weir_data = {
    "id": "DV_1471", 
    "branchId": "458", 
    "chainage": 106.277467, 
    "allowedFlowDir": FlowDirection.both, 
    "crestLevel": 0.400, 
    "crestWidth": 40.000, 
    "corrCoeff": 1.000, 
    "useVelocityHeight": True
}
weir = Weir(**weir_data)

# Add the weir to the model
structure_file.structure.append(weir)

print(f"Number of structures: {len(structure_file.structure)}")


### Adjusting some parameters of the model

In [ ]:
model.physics.backgroundsalinity = 30 # [ppt]
model.physics.backgroundwatertemperature = 6 # [°C]
model.time.dtuser = 900 # [s]

## Saving the model
If we save the model now, it will overwrite the current model files.
So let's save it in a different location.

In [ ]:
print(f"The original model is located at {model.filepath}")

model.filepath = demo_data_folder / "save" / "FlowFM.mdu"

# Set recurse to True to make sure that not only the MDU file is saved, but also the child model files.
model.save(recurse=True)

print(f"The saved model is located at {model.filepath}")

### Saving a "child" model
It is also possible to save individual child model files, such as the cross section definition file.

In [ ]:
crossdef_file = model.geometry.crossdeffile
assert crossdef_file is not None

crossdef_file.filepath = demo_data_folder / "save_crossdef" / "crsdef.ini"
crossdef_file.save()
